# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [2]:
import getpass
import pyspark
from datetime import datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers
import pickle

%load_ext autoreload
%autoreload 2

## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-fouche>

In [4]:
# init spark session
spark = SparkSession(sc)

## Loading the data

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe:

In [5]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

In [6]:
# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

In [7]:
metadata.show(5)

+----------------+----------+---------+---------+------+
|            name|station_ID|     long|      lat|height|
+----------------+----------+---------+---------+------+
|       Bucuresti|         2|26.074412| 44.44677|     0|
|          Calais|         3| 1.811446|50.901549|     0|
|      Canterbury|         4| 1.075329|51.284212|     0|
|          Exeter|         5|-3.543547|50.729172|     0|
|Fideris, Bahnhof|         7| 9.733756|46.922368|   744|
+----------------+----------+---------+---------+------+
only showing top 5 rows



We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [8]:
metadata.count()

25935

In [9]:
# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

In [10]:
# convert to pandas dataframe
pandas_df = metadata.toPandas()
# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

In [11]:
# pandas_df.distance_to_zh.max()

In [12]:
# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)
# create dict of stations from pandas dataframe
stations = pandas_df.set_index('station_ID').to_dict('index')

### Cleaning main dataset

In [13]:
# load full data
# raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [14]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [15]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [16]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', helpers.get_weekday(df.date))

In [17]:
# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [18]:
# there is still 51'571'541 rows in zurich area
# df.count()

In [49]:
# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '2017-12-10 00:00:00')

In [109]:
df.limit(100).toPandas()

,date,trip_id,transport_type,train_id,train_type,additional_trip,trip_failed,stop_name,stop_id,schedule_arrival,real_arrival,arr_forecast_status,schedule_dep,real_dep,dep_forecast_status,no_stop_here,weekday
0,2018-01-02 00:00:00,85:11:10:002,Zug,10,EC,False,False,Zürich HB,8503000,2018-01-02 21:51:00,2018-01-02 21:53:00,GESCHAETZT,None,None,PROGNOSE,False,1
1,2018-01-02 00:00:00,85:11:10578:001,Zug,10578,IC,True,False,Zürich HB,8503000,2018-01-02 17:19:00,2018-01-02 17:20:00,GESCHAETZT,None,None,PROGNOSE,False,1
2,2018-01-02 00:00:00,85:11:11:001,Zug,11,EC,False,False,Zürich HB,8503000,None,None,PROGNOSE,2018-01-02 06:09:00,2018-01-02 06:10:00,GESCHAETZT,False,1
3,2018-01-02 00:00:00,85:11:12:001,Zug,12,EC,False,False,Zürich HB,8503000,2018-01-02 10:51:00,2018-01-02 10:51:00,GESCHAETZT,None,None,PROGNOSE,False,1
4,2018-01-02 00:00:00,85:11:1251:001,Zug,1251,IC,False,False,Zürich HB,8503000,2018-01-02 07:00:00,2018-01-02 06:59:00,GESCHAETZT,None,None,PROGNOSE,False,1
5,2018-01-02 00:00:00,85:11:1252:002,Zug,1252,IC,False,False,Zürich HB,8503000,None,None,PROGNOSE,2018-01-02 21:36:00,2018-01-02 21:38:00,GESCHAETZT,False,1
6,2018-01-02 00:00:00,85:11:1253:001,Zug,1253,IC,False,False,Zürich HB,8503000,2018-01-02 08:00:00,2018-01-02 08:01:00,GESCHAETZT,2018-01-02 08:07:00,2018-01-02 08:08:00,GESCHAETZT,False,1
7,2018-01-02 00:00:00,85:11:1256:001,Zug,1256,IC,False,False,Zürich HB,8503000,2018-01-02 17:53:00,2018-01-02 17:54:00,GESCHAETZT,2018-01-02 18:00:00,2018-01-02 18:01:00,GESCHAETZT,False,1
8,2018-01-02 00:00:00,85:11:1260:001,Zug,1260,IC,False,False,Zürich HB,8503000,None,None,PROGNOSE,2018-01-02 21:00:00,2018-01-02 21:00:00,GESCHAETZT,False,1
9,2018-01-02 00:00:00,85:11:1261:001,Zug,1261,IC,False,False,Zürich HB,8503000,2018-01-02 21:26:00,2018-01-02 21:32:00,GESCHAETZT,None,None,PROGNOSE,False,1


In [104]:
# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

In [105]:
df2 = df2.where((df2.schedule_dep >= df2.schedule_arrival) | functions.col('schedule_arrival').isNull() | functions.col('schedule_dep').isNull())

## Modeling the network

### From stops to trips

In [106]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [107]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep', 'transport_type', 'train_type', 'arr_forecast_status', 'weekday', 'real_arrival']).orderBy('trip_id')

In [108]:
trips.limit(50).toPandas()

,trip_id,date,schedule_time,stop_id,stop_type,schedule_arrival,schedule_dep,transport_type,train_type,arr_forecast_status,weekday,real_arrival
0,81:81____:401:000,2018-01-23 00:00:00,2018-01-24 09:05:00,8503000,last,2018-01-24 09:05:00,None,Zug,NJ,UNBEKANNT,1,None
1,81:81____:401:000,2018-01-22 00:00:00,2018-01-23 09:05:00,8503000,last,2018-01-23 09:05:00,None,Zug,NJ,UNBEKANNT,0,None
2,81:81____:401:000,2018-01-24 00:00:00,2018-01-25 09:05:00,8503000,last,2018-01-25 09:05:00,None,Zug,NJ,UNBEKANNT,2,None
3,85:11:10563:001,2018-01-27 00:00:00,2018-01-27 09:29:00,8503000,first,None,2018-01-27 09:29:00,Zug,IC,PROGNOSE,5,None
4,85:11:10565:001,2018-01-20 00:00:00,2018-01-20 10:29:00,8503000,first,None,2018-01-20 10:29:00,Zug,IC,PROGNOSE,5,None
5,85:11:10565:001,2018-01-27 00:00:00,2018-01-27 10:29:00,8503000,first,None,2018-01-27 10:29:00,Zug,IC,PROGNOSE,5,None
6,85:11:10578:001,2018-01-02 00:00:00,2018-01-02 17:19:00,8503000,last,2018-01-02 17:19:00,None,Zug,IC,GESCHAETZT,1,2018-01-02 17:20:00
7,85:11:10578:002,2018-01-14 00:00:00,2018-01-14 17:19:00,8503000,last,2018-01-14 17:19:00,None,Zug,IC,GESCHAETZT,6,2018-01-14 17:20:00
8,85:11:10578:002,2018-01-07 00:00:00,2018-01-07 17:19:00,8503000,last,2018-01-07 17:19:00,None,Zug,IC,GESCHAETZT,6,2018-01-07 17:21:00
9,85:11:10578:002,2018-01-28 00:00:00,2018-01-28 17:19:00,8503000,last,2018-01-28 17:19:00,None,Zug,IC,GESCHAETZT,6,2018-01-28 17:19:00


In [100]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))
trips2 = trips2.select("*", functions.lag("arr_forecast_status").over(w).alias("next_arr_forecast_status"))
trips2 = trips2.select("*", functions.lag("real_arrival").over(w).alias("next_real_arrival"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [72]:
trips2.count()

7152183

In [24]:
#trips2.where(trips2.stop_type=='first').count()
#trips2.where(trips2.stop_type=='last').count()
#trips2.where(trips2.stop_type=='mid').count()

In [101]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type))

In [102]:
# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

In [86]:
# trips4.select('stop_id', 'next_sid').distinct().count()
# gives 6606

# trips3.select('stop_id', 'next_sid').distinct().count()
# gives 8557

In [103]:
trips4.where(trips4.schedule_dep > trips4.next_sched_dep).count()

158255

In [88]:
trips4.where(trips4.schedule_dep > trips4.next_sched_dep).limit(100).toPandas()

,trip_id,date,schedule_time,stop_id,stop_type,schedule_arrival,schedule_dep,transport_type,train_type,arr_forecast_status,...,real_arrival,next_tid,next_time,next_sid,next_type,next_sched_arr,next_sched_dep,next_arr_forecast_status,next_real_arrival,is_valid
0,85:3849:107259-03004-1,2018-01-26 00:00:00,2018-01-26 07:09:00,8576200,mid,2018-01-26 07:09:00,2018-01-26 07:10:00,Tram,None,PROGNOSE,...,2018-01-26 07:10:00,85:3849:107259-03004-1,2018-01-26 07:09:00,8576199,mid,2018-01-26 07:09:00,2018-01-26 07:09:00,PROGNOSE,2018-01-26 07:09:00,true
1,85:3849:107259-03004-1,2018-01-19 00:00:00,2018-01-19 07:09:00,8576200,mid,2018-01-19 07:09:00,2018-01-19 07:10:00,Tram,None,PROGNOSE,...,2018-01-19 07:09:00,85:3849:107259-03004-1,2018-01-19 07:09:00,8576199,mid,2018-01-19 07:09:00,2018-01-19 07:09:00,PROGNOSE,2018-01-19 07:09:00,true
2,85:3849:107259-03004-1,2018-01-12 00:00:00,2018-01-12 07:09:00,8576200,mid,2018-01-12 07:09:00,2018-01-12 07:10:00,Tram,None,PROGNOSE,...,2018-01-12 07:09:00,85:3849:107259-03004-1,2018-01-12 07:09:00,8576199,mid,2018-01-12 07:09:00,2018-01-12 07:09:00,PROGNOSE,2018-01-12 07:08:00,true
3,85:3849:107259-03004-1,2018-01-05 00:00:00,2018-01-05 07:09:00,8576200,mid,2018-01-05 07:09:00,2018-01-05 07:10:00,Tram,None,PROGNOSE,...,2018-01-05 07:09:00,85:3849:107259-03004-1,2018-01-05 07:09:00,8576199,mid,2018-01-05 07:09:00,2018-01-05 07:09:00,PROGNOSE,2018-01-05 07:08:00,true
4,85:3849:109049-03010-1,2018-01-26 00:00:00,2018-01-26 16:08:00,8591244,mid,2018-01-26 16:08:00,2018-01-26 16:09:00,Tram,None,PROGNOSE,...,2018-01-26 16:10:00,85:3849:109049-03010-1,2018-01-26 16:08:00,8591255,mid,2018-01-26 16:08:00,2018-01-26 16:08:00,PROGNOSE,2018-01-26 16:09:00,true
5,85:3849:109049-03010-1,2018-01-19 00:00:00,2018-01-19 16:08:00,8591244,mid,2018-01-19 16:08:00,2018-01-19 16:09:00,Tram,None,PROGNOSE,...,2018-01-19 16:09:00,85:3849:109049-03010-1,2018-01-19 16:08:00,8591255,mid,2018-01-19 16:08:00,2018-01-19 16:08:00,PROGNOSE,2018-01-19 16:09:00,true
6,85:3849:109049-03010-1,2018-01-12 00:00:00,2018-01-12 16:08:00,8591244,mid,2018-01-12 16:08:00,2018-01-12 16:09:00,Tram,None,PROGNOSE,...,2018-01-12 16:10:00,85:3849:109049-03010-1,2018-01-12 16:08:00,8591255,mid,2018-01-12 16:08:00,2018-01-12 16:08:00,PROGNOSE,2018-01-12 16:09:00,true
7,85:3849:109049-03010-1,2018-01-05 00:00:00,2018-01-05 16:08:00,8591244,mid,2018-01-05 16:08:00,2018-01-05 16:09:00,Tram,None,PROGNOSE,...,2018-01-05 16:09:00,85:3849:109049-03010-1,2018-01-05 16:08:00,8591255,mid,2018-01-05 16:08:00,2018-01-05 16:08:00,PROGNOSE,2018-01-05 16:08:00,true
8,85:3849:109510-03011-1,2018-01-26 00:00:00,2018-01-26 12:05:00,8591180,mid,2018-01-26 12:05:00,2018-01-26 12:06:00,Tram,None,PROGNOSE,...,2018-01-26 12:05:00,85:3849:109510-03011-1,2018-01-26 12:05:00,8591429,mid,2018-01-26 12:05:00,2018-01-26 12:05:00,PROGNOSE,2018-01-26 12:05:00,true
9,85:3849:109510-03011-1,2018-01-19 00:00:00,2018-01-19 12:05:00,8591180,mid,2018-01-19 12:05:00,2018-01-19 12:06:00,Tram,None,PROGNOSE,...,2018-01-19 12:05:00,85:3849:109510-03011-1,2018-01-19 12:05:00,8591429,mid,2018-01-19 12:05:00,2018-01-19 12:05:00,PROGNOSE,2018-01-19 12:05:00,true


### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [ ]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [ ]:
regenerate_models = False
models = []
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = helpers.model_network(trips4, date)
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

In [ ]:
# load the networks from the pickles
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)

In [ ]:
models[0][list(models[0].keys())[0]]

## Prediction / Regression

Deux idées : Le but c'est de prévoir le retard ou l'avance d'un départ ou d'une arrivée pour un arrêt précis à un moment précis et un trip précis.

- Faire une vraie regression ou n'importe quel algo pour déterminer le retard. On utilise comme features : l'arrêt (catégorie), latitude longitude (peut être mettre ensemble en tuple), jour precis ou jour de la semaine ?, le type de transport (bus-train ...), le trip id ou redondant ???, 

- Faire une moyenne, à un arrêt, un trip, un jour, une heure la moyenne de retard qu'il a eu dans ses conditions ... 

On va avoir besoin des trips id dans le dico histoire de pouvoir lier le graph des arrêts avec le réél trajet, quel bus/train fait cet edge.

### Preprocessing :

In [87]:
trips5 = trips4.drop('stop_type', 'next_type', 'is_valid', 'arr_forcast_status', 
                     'schedule_time', 'schedule_arrival', 'next_sched_dep', 
                     'next_time', 'next_tid', 'real_arrival', 'arr_forecast_status')

In [88]:
@functions.udf
def keep_time(date):
    return date.split(' ')[1]

In [89]:
trips5 = trips5.withColumn('schedule_dep', keep_time(trips4.schedule_dep))
trips5 = trips5.withColumn('next_sched_arr', keep_time(trips4.next_sched_arr))
trips5 = trips5.withColumn('next_real_arrival', keep_time(trips4.next_real_arrival))

In [90]:
@functions.udf
def create_interval(date):
    # rush_hour = 6/9 17/19 = 0
    # not_rush_hour = 0/6 9/17 19/24  = 1
    if (date >= '06:00:00' and date <= '09:00:00') or (date >= '17:00:00' and date <= '19:00:00'):
        return 0
    else:
        return 1

In [91]:
# We compute the arrival delay of each stop station
df3 = trips5.withColumn("delay_arrival", 
                     functions.unix_timestamp('next_real_arrival', 'HH:mm:ss') -
                     functions.unix_timestamp('next_sched_arr', 'HH:mm:ss'))

# We create time interval in order to differentiate rush hour and other time of the day
# We have to try different combinations of interval time (that gave best prediction)
df3 = df3.withColumn('arrival_interval', create_interval(df3.next_sched_arr))

In [157]:
# We create 2 dataframes, one for the arrival (without the first departure stations)
arrival_df = df3.filter(df3.next_arr_forecast_status == "GESCHAETZT")
# only time of train (Zug) was evaluated
arrival_df = arrival_df.drop('next_arr_forecast_status', 'transport_type')

arrival_df = arrival_df.filter(arrival_df['delay_arrival'] <= 600)

In [149]:
arrival_df.limit(10).toPandas()

,trip_id,date,stop_id,schedule_dep,train_type,weekday,next_sid,next_sched_arr,next_real_arrival,delay_arrival,arrival_interval,train_type_cat
0,85:11:13752:001,2018-01-28 00:00:00,8502221,02:12:00,SN,6,8502222,02:16:00,02:18:00,120,1,2
1,85:11:13752:001,2018-01-28 00:00:00,8502229,02:07:00,SN,6,8502221,02:12:00,02:13:00,60,1,2
2,85:11:13752:001,2018-01-28 00:00:00,8502220,02:06:00,SN,6,8502229,02:07:00,02:10:00,180,1,2
3,85:11:13752:001,2018-01-28 00:00:00,8503001,02:01:00,SN,6,8502220,02:06:00,02:08:00,120,1,2
4,85:11:13752:001,2018-01-28 00:00:00,8503020,01:59:00,SN,6,8503001,02:01:00,02:02:00,60,1,2
5,85:11:13752:001,2018-01-28 00:00:00,8503000,01:57:00,SN,6,8503020,01:59:00,01:59:00,0,1,2
6,85:11:13752:001,2018-01-28 00:00:00,8503003,01:52:00,SN,6,8503000,01:55:00,01:54:00,-60,1,2
7,85:11:13752:001,2018-01-27 00:00:00,8502221,02:12:00,SN,5,8502222,02:16:00,02:18:00,120,1,2
8,85:11:13752:001,2018-01-27 00:00:00,8502229,02:07:00,SN,5,8502221,02:12:00,02:13:00,60,1,2
9,85:11:13752:001,2018-01-27 00:00:00,8502220,02:06:00,SN,5,8502229,02:07:00,02:10:00,180,1,2


In [158]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer

train_type_Indexer = StringIndexer(inputCol="train_type", outputCol="train_type_cat")
train_type_model = train_type_Indexer.fit(arrival_df)
arrival_df = train_type_model.transform(arrival_df)

arrival_df = arrival_df.withColumn('weekday', arrival_df['weekday'].cast('int'))
arrival_df = arrival_df.withColumn('train_type_cat', arrival_df['train_type_cat'].cast('int'))

arrival_df_test = arrival_df.filter(arrival_df.weekday < 5)
arrival_df_test = arrival_df_test.withColumn('delay_arrival', arrival_df_test.delay_arrival/60)

vec_ass = VectorAssembler(inputCols=['delay_arrival', 'train_type_cat'], outputCol="features")
kmeans_df = vec_ass.transform(arrival_df_test)
kmean = KMeans(k=7)
model_km = kmean.fit(kmeans_df)
kmeans_df = model_km.transform(kmeans_df)

In [159]:
kmeans_df = kmeans_df.withColumn('timestamp', functions.unix_timestamp('next_sched_arr',  'HH:mm:ss'))

In [160]:
mean_ = kmeans_df.groupBy('prediction').mean('timestamp').collect()
print(mean_)

[Row(prediction=1, avg(timestamp)=82710.0), Row(prediction=6, avg(timestamp)=45273.397385415024), Row(prediction=3, avg(timestamp)=47625.75884032506), Row(prediction=5, avg(timestamp)=46902.68088697921), Row(prediction=4, avg(timestamp)=45347.890784982934), Row(prediction=2, avg(timestamp)=46109.246196136126), Row(prediction=0, avg(timestamp)=47652.756985954475)]


In [161]:
for i in mean_:
    print(i['prediction'])
    print(datetime.datetime.fromtimestamp(i['avg(timestamp)']).strftime("%H:%M:%S"))

1
23:58:30
6
13:34:33
3
14:13:45
5
14:01:42
4
13:35:47
2
13:48:29
0
14:14:12


## PREDICTION / pandas and pyspark

In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import pandas as pd

In [ ]:
le_train_type = preprocessing.LabelEncoder()
le_train_type.fit(arrival_pd_df['train_type'])
# see the class
#list(le_train_type.classes_)

In [ ]:
arrival_pd_df['train_type_cat'] = le_train_type.transform(arrival_pd_df['train_type'])
# we can do the inverse
# arrival_pd_df['train_type'] = le_train_type.inverse_transform(arrival_pd_df['train_type_cat'])

In [ ]:
arrival_pd_df

In [ ]:
arrival_pd_df.weekday = arrival_pd_df.weekday.astype(int)
arrival_pd_df.arrival_interval = arrival_pd_df.arrival_interval.astype(int)
arrival_pd_df.arrival_interval = arrival_pd_df.arrival_interval.astype(int)

arrival_pd_df.dtypes

In [ ]:
week_df = arrival_pd_df[arrival_pd_df.weekday == 0]
week_df.delay_arrival = week_df.delay_arrival/60
week_df

In [ ]:
lin_regr = LinearRegression()
lin_regr.fit(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']], week_df.delay_arrival)
pred = lin_regr.predict(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']])
lin_regr.score(week_df[['stop_id', 'next_sid', 'arrival_interval', 'train_type_cat']],  week_df.delay_arrival)

In [ ]:
week_df['pred'] = pred
week_df

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

trip_Indexer = StringIndexer(inputCol="trip_id", outputCol="trip_id_index")
trip_model = trip_Indexer.fit(arrival_df)

arrival_df = trip_model.transform(arrival_df)

In [ ]:
arrival_df = arrival_df.drop('trip_id', 'transport_type')

In [ ]:
arrival_df = arrival_df.withColumn('weekday', arrival_df['weekday'].cast('int'))
arrival_df = arrival_df.withColumn('arrival_interval', arrival_df['arrival_interval'].cast('int'))

In [ ]:
arrival_df = arrival_df.withColumn('delay_arrival', arrival_df['delay_arrival'] / 60)

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler


vec_ass = VectorAssembler(inputCols=['stop_id','weekday','arrival_interval','trip_id_index','transport_type_index'], outputCol="features")

#arrival_df = vec_ass.transform(arrival_df)
lr = LinearRegression(maxIter=5, regParam=0.0, solver="normal", 
                      labelCol='delay_arrival')
model = lr.fit(arrival_df)

test = model.transform(arrival_df)

In [ ]:
test = test.withColumn('evaluation', test['delay_arrival'] - test['prediction'])

In [ ]:
test.describe('evaluation').show()

In [ ]:
test.describe('delay_arrival').show()

In [ ]:
test.describe('prediction').show()

In [ ]:
test_df = arrival_df.limit(50).toPandas()
test_df